各階段設計思路
最後更新：2022/07/10

# **階段二、設計系統(初始設定)**

接下來在系統的運行中，有些部分是需要先定義的。  


> 例如函式、數值及空串列。


但如果每次都是要用到的時候才給定義的話，版面或許會有些凌亂。  
因此我選擇將它納入進最前面的部分，也就是程式運行篩選迴圈前，就先弄完定義，這樣空間應該會整潔不少。  
下列我們看系統初始設定的部分。

首先，我們在運行時，會使用到一些原本Python不會預設附帶的套件，因此我們需要利用pip指令來安裝，如下。  


```
!pip install twstock
```


以Google Colab為例，我們會需要額外安裝twstock。  
至於一般有預設的套件則是直接import即可，不需重複安裝。  
由於套件名稱有點冗長，於是我也另外給定對應的代號。

In [2]:
#匯入函式資料庫
#!pip install twstock
import twstock as t 
import pandas as p 
from bs4 import BeautifulSoup 
import requests 
import time as ti
from datetime import datetime
from pytz import timezone   

接下來是定義函式的部分。  
這個系統中我們運用到兩個函式，一個是**`網頁查詢使用`**，另一個是**`Line Notify通知`**。  
由於本身對於api串接的熟悉度不是很好，因此這邊參考他人的[程式代碼](https://jeffwen0105.com/python_linenotify/)。

In [9]:
def web(url):#定義網頁查詢函式
    source = requests.get(url, headers={'Connection':'close'}) #連線到指定的網站
    #因為股票要篩選好幾千個，若我們一直開啟新網頁而不關掉已使用完畢的頁面，電腦資源會被浪費
    soup = BeautifulSoup(source.content, 'lxml', from_encoding='utf-8') #讀取這個網頁的內容
    return soup #最終結果:回傳網頁內容

def post_data(message, token):#Line notify傳訊息
    try:
        url = "https://notify-api.line.me/api/notify"#api網址
        headers = {
            'Authorization': f'Bearer {token}'
        }
        payload = {
            'message': message
        }
        response = requests.request(
            "POST",
            url,
            headers=headers,
            data=payload
        )
        if response.status_code == 200:
            print(f"Success -> {response.text}")
    except Exception as _:
        print(_)#狀態

接下來是取得最新股票資料。  
在這邊我們利用**`pandas套件`**去讀取網頁內容，因上市及上櫃股為不同網頁因此這邊要取分開得兩次資料。  
因為我們只需要 "有價證券代號及名稱" 的這部分資訊，因此我們單獨把這部分拉來製表後，再替代掉原先的表格。  
接下來我們只需要重複上市股的取得動作，但把網址改成上櫃股的網址，這樣我們就完成取得上市股跟上櫃股的最新清單步驟了！

In [4]:
#####取得上市股#####
data = p.read_html('http://isin.twse.com.tw/isin/C_public.jsp?strMode=2', encoding='big5hkscs', header=0)[0]
data = p.DataFrame(data["有價證券代號及名稱"])
data = data["有價證券代號及名稱"].astype(str)

#####取得上櫃股#####
data2 = p.read_html('http://isin.twse.com.tw/isin/C_public.jsp?strMode=4', encoding='big5hkscs', header=0)[0]
data2 = p.DataFrame(data2["有價證券代號及名稱"])
data2 = data2["有價證券代號及名稱"].astype(str)
print(data,"\n=====\n",data2)

0                   股票
1              1101　台泥
2              1102　亞泥
3              1103　嘉泥
4              1104　環泥
             ...      
30574    01003T　兆豐新光R1
30575    01004T　土銀富邦R2
30576    01007T　兆豐國泰R2
30577    01009T　王道圓滿R1
30578    01010T　京城樂富R1
Name: 有價證券代號及名稱, Length: 30579, dtype: object 
=====
 0              上櫃認購(售)權證
1       70202P　力旺中信17售01
2       70217P　威剛元富17售01
3       70218P　群聯元富17售03
4       70236P　鈊象元富17售03
              ...       
7928       01017S　93中信貸d
7929      01109S　051中租賃A
7930      01110S　051中租賃B
7931      01111S　081中租賃A
7932      01112S　081中租賃B
Name: 有價證券代號及名稱, Length: 7933, dtype: object


接下來是一些基本定義。  
因為後續判斷條件中，我在作者的判斷條件上另外多加**` 是否為當沖個股 `**的判斷條件。  
因此我也在前期就先給定好，後續可以直接調用。

代數預設指定是針對部分後續會使用到的代數預先指定*。

> *但經1.7版改版後，因調整自檢查機制，因此這部分的某些數值已經放在迴圈中



In [6]:
#####取得可當沖股#####
TW50 = [1101, 1216, 1301, 1303, 1326, 1590, 2002, 2207, 2227, 2303, 2308, 2317, 2327, 2330, 2352, 2357, 2379, 2382, 2395, 2408, 2409, 2412, 2454, 2603, 2609, 2610, 2615, 2801, 2880, 2881, 2882, 2883, 2884, 2885, 2886, 2887, 2891, 2892, 2912, 3008, 3034, 3037, 3045, 3711, 4904, 5871, 5876, 5880, 6415, 6505, 8046, 8454, 9910]
RICH50 = [1565, 1785, 3081, 3105, 3152, 3211, 3218, 3227, 3228, 3260, 3264, 3293, 3324, 3374, 3529, 3552, 3680, 3707, 4105, 4123, 4128, 4736, 4966, 5274, 5289, 5347, 5351, 5371, 5425, 5483, 6104, 6121, 6138, 6147, 6182, 6274, 6411, 6488, 6510, 6547, 6548, 6561, 6732, 6741, 8069, 8086, 8255, 8299, 8358, 8436]
MID100A = [1102,1210,1227,1229,1402,1434,1440,1451,1476,1477,1504,1560,1605,1717,1718,1722,1802,1907,2006,2014,2023,2027,2049,2105,2201,2301,2312,2313,2324,2337,2344,2347,2353,2354,2356,2360,2371,2376,2377,2385,2441,2449,2458,2474,2478,2492,2498,2542,2606,2618,2633,2637,2809,2812,2834,2845,2888,2889,2890,2915,3023,3035,3036,]
MID100B = [3044,3051,3189,3231,3406,3443,3481,3532,3533,3653,3661,3702,3714,4137,4919,4938,4958,4961,5269,5522,6005,6116,6176,6213,6239,6271,6409,6531,6592,6669,6719,6770,6781,8464,9904,9914,9917,9921,9941,9945]
Ulti_list = RICH50  + MID100A + MID100B + TW50 

#####創建空串列#####
stocklist_data = []#上市篩選股票清單
stocklist_data2 = []#上櫃篩選股票清單
stocklist = []#篩選股票清單
stock28list = []#符合條件清單
amu28list = []#符合精選條件清單
ulti28list = []#符合當沖精選條件清單

#####代數預設指定#####
eps_check = 0 #eps資料檢查
divi_check = 0 #dividend資料檢查
self_divi_check1 = 0 #dividend資料自檢查1
self_divi_check2 = 0 #dividend資料自檢查2
self_eps_check1 = 0 #eps資料自檢查1
self_eps_check2 = 0 #eps資料自檢查2
characters = "%," #設定特定字符
rps_result = str('查無計算所需資料')#rps_result初始給值
data_check = 0 #上市篩選索引檢查
data2_check = 0 #上櫃篩選索引檢查
TPE_time = timezone('Asia/Taipei') #本機時間時區指定

因為股票會有新上市或新下市的情況，同時系統抓到的股票清單內，會含有我不想要抓到的東西，**`例如:權證或ETF等 `**。    
因此我讓系統自己從抓到的股票清單中，找尋清單中最開始的股票跟最後的股票內容，以此來確保系統篩選的都是股票，也保證清單是最新的。  

In [7]:
#####抓找上市篩選股票清單#####
for x in data.iloc:#個別抓資料
    try:
        if x == "股票":#找到"股票"索引
            data_check = 1 #上市篩選索引開始
        if x == "上市認購(售)權證":#找到"上市認購(售)權證"索引
            data_check = 0 #上市篩選索引結束
        if data_check == 1:#如果上市篩選索引開始
            stockid = x
            stocklist_data.append(stockid)#匯入清單
    except:
        continue
stocklist_data.remove("股票")#清除清單中的"股票"

#####抓找上櫃篩選股票清單#####
for z in data2.iloc:#個別抓資料
    try:
        if z == "股票":#找到"股票"索引
            data2_check = 1 #上櫃篩選索引開始
        if z == "特別股":#找到"特別股"索引
            data2_check = 0 #上櫃篩選索引結束
        if data2_check == 1:#如果上櫃篩選索引開始
            stockid = z
            stocklist_data2.append(stockid)#匯入清單
    except:
        continue
stocklist_data2.remove("股票")#清除清單中的"股票"
print(stocklist_data,"\n=====\n",stocklist_data2)

['1101\u3000台泥', '1102\u3000亞泥', '1103\u3000嘉泥', '1104\u3000環泥', '1108\u3000幸福', '1109\u3000信大', '1110\u3000東泥', '1201\u3000味全', '1203\u3000味王', '1210\u3000大成', '1213\u3000大飲', '1215\u3000卜蜂', '1216\u3000統一', '1217\u3000愛之味', '1218\u3000泰山', '1219\u3000福壽', '1220\u3000台榮', '1225\u3000福懋油', '1227\u3000佳格', '1229\u3000聯華', '1231\u3000聯華食', '1232\u3000大統益', '1233\u3000天仁', '1234\u3000黑松', '1235\u3000興泰', '1236\u3000宏亞', '1256\u3000鮮活果汁-KY', '1301\u3000台塑', '1303\u3000南亞', '1304\u3000台聚', '1305\u3000華夏', '1307\u3000三芳', '1308\u3000亞聚', '1309\u3000台達化', '1310\u3000台苯', '1312\u3000國喬', '1313\u3000聯成', '1314\u3000中石化', '1315\u3000達新', '1316\u3000上曜', '1319\u3000東陽', '1321\u3000大洋', '1323\u3000永裕', '1324\u3000地球', '1325\u3000恆大', '1326\u3000台化', '1337\u3000再生-KY', '1338\u3000廣華-KY', '1339\u3000昭輝', '1340\u3000勝悅-KY', '1341\u3000富林-KY', '1342\u3000八貫', '1402\u3000遠東新', '1409\u3000新纖', '1410\u3000南染', '1413\u3000宏洲', '1414\u3000東和', '1416\u3000廣豐', '1417\u3000嘉裕', '1418\u3000東華', '1419\u3000新紡',

由於上市上櫃是兩個清單，因此需要先分別抓取後才能再彙整成一份清單。
整合完畢後也通知使用者，提醒使用者初始化完成。

In [10]:
#####整合上市上櫃篩選股票清單#####
stocklist = stocklist_data + stocklist_data2#整合

#####Line Notify通知#####
if __name__ == "__main__":
  token = "b7wgZypudip09Ka7N3NrW56AeRn5xYE9lEOpwfK7bDs" # Token
  message = {"\n⚙系統初始化設定完成\n🤖開始篩選股票\n提醒您，相關操作說明請詳閱參閱連結說明喔!\n bacons.cc/d/gvvw"}     # 要發送的訊息
  post_data(message, token)

print("系統設定完成...\n開始篩選股票\n===============================")
print("實時符合當沖精選條件的股票:")

Success -> {"status":200,"message":"ok"}
系統設定完成...
開始篩選股票
實時符合當沖精選條件的股票:
